# CF 를 위한 코드 ZIP 파일 패키징 (관리자용)
---

# 0. 환경 설정

### [중요] bucket 이름은 반드시 바꾸어 주세요. !!!

aws s3 cp my-asset.zip s3://ee-assets-prod-us-east-1/modules/c16b965656fb4eba8676d6f0ac759300/v1/my-asset.zip

In [1]:
bucket = 'ee-assets-prod-us-east-1'
region = 'us-east-1'
project_prefix = 'modules/c16b965656fb4eba8676d6f0ac759300/v1/Lex-KR_BankingBot'

### 코드 폴더 

In [2]:
Biz_DDB_Code_Folder = 'Artifact/Lambda-BizLogic-DynamoDB-Code'
Biz_Logic_Code_Folder = 'Artifact/Lambda-BizLogic-BankingService-Code'
Lex_Artifact_Folder = 'Artifact/Lambda-LexArtifact-Export-Artifact'

# 1. CF YAML 파일 및 준비 파일 업로드

## 1.1 YAML 파일의  Bucket 및 Prefix 경로 수정

### YAML 파일
[중요] `Deploy-Package/CF_KRBankingBot.yaml` 파일을 위의 버켓 이름과 project_prefix 에 맞게 수정 해주세요.

![BankingYAML.png](img/BankingYAML.png)

## 1.2 YAML 파일 S3 업로드

In [3]:
CFFolder = "Deploy-Package"

In [4]:
import os

prefix = "CF-YAML-File"
file_name = 'CF_KRBankingBot.yaml'

s3_file_path = 's3://' + os.path.join(bucket, project_prefix, prefix, file_name)
local_file_path = os.path.join(CFFolder, file_name)

print("s3_file_path: ", s3_file_path)
print("local_file_path: ", local_file_path)


s3_file_path:  s3://ee-assets-prod-us-east-1/modules/c16b965656fb4eba8676d6f0ac759300/v1/Lex-KR_BankingBot/CF-YAML-File/CF_KRBankingBot.yaml
local_file_path:  Deploy-Package/CF_KRBankingBot.yaml


In [5]:
%%sh -s "{local_file_path}" "{s3_file_path}"

export AWS_ACCESS_KEY_ID=<dummy> # 키를 입력 하세요
export AWS_SECRET_ACCESS_KEY=<dummy> # Secret Key 입력하세요
export AWS_SESSION_TOKEN=<dummy> # Token 입력하세요 
export AWS_DEFAULT_REGION=<dummy> # region 입력 하세요  (예: us-east-1)



local_file_path=$1
s3_file_path=$2

aws s3 cp $local_file_path $s3_file_path 
aws s3 ls $s3_file_path 

upload: Deploy-Package/CF_KRBankingBot.yaml to s3://ee-assets-prod-us-east-1/modules/c16b965656fb4eba8676d6f0ac759300/v1/Lex-KR_BankingBot/CF-YAML-File/CF_KRBankingBot.yaml
2022-02-16 11:22:06       8817 CF_KRBankingBot.yaml


## 1.3 YAML S3 경로

In [6]:
print(f"https://{bucket}.s3.{region}.amazonaws.com/{project_prefix}/{prefix}/{file_name}")

https://ee-assets-prod-us-east-1.s3.us-east-1.amazonaws.com/modules/c16b965656fb4eba8676d6f0ac759300/v1/Lex-KR_BankingBot/CF-YAML-File/CF_KRBankingBot.yaml


# 2. DDB 레코드 입력 람다 코드 준비

In [7]:
DDBInsertFolder = "Deploy-Package/Lambda-BizLogic-DynamoDB"
DDBInsertCodeZip = "DDBInsert.zip"

### (1) 코드 파일, 의존성 패키지를 압축한 Zip 파일 생성

#### Shell 명령어 설명
- DDBInsertCodeZip 파일이 존재하면 삭제
- 의존 패키지가 저장된 package 폴더가 있으면 삭제
- 의존성 패키지 requests 설치
- 의존성 패키지를 DDBInsertCodeZip 파일에 추가
- DDBInsertCodeZip 파일에 lambda_function.py cfnresponse.py BankingBotData.json 파일 추가

In [8]:
%%sh -s {DDBInsertFolder} {DDBInsertCodeZip} {Biz_DDB_Code_Folder}
DynamoTableInsertFolder=$1
DBInsertCodeZip=$2
Biz_DDB_Code_Folder=$3

echo $DynamoTableInsertFolder
echo $DBInsertCodeZip
echo $Biz_DDB_Code_Folder

# 코드 있는 폴더로 이동
cd $DynamoTableInsertFolder

# 코드를 압축한 zip 파일 있으면 삭제
if test -f $DBInsertCodeZip; then
    rm $DBInsertCodeZip
    echo "Existing Code Zip file is deleted"
fi

# package 폴더 있으면 삭제
if [ -d "package" ]; then
    rm -rf package
    echo "existing package folder is deleted" 
fi    

# requests 패키지 다운로드
pip install --target ./package requests

cd package
# 패키지 폴더 내용 압축
zip -r ../$DBInsertCodeZip .
cd ..

echo 'Current Director: '
echo $PWD
# 코드 파일 및 데이터 파일을 압축 파일에 추가
zip -g -j $DBInsertCodeZip ../../$Biz_DDB_Code_Folder/*.py ../../$Biz_DDB_Code_Folder/*.json
echo "Completed"


Deploy-Package/Lambda-BizLogic-DynamoDB
DDBInsert.zip
Artifact/Lambda-BizLogic-DynamoDB-Code
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Using cached urllib3-1.26.8-py2.py3-none-any.whl (138 kB)
  Using cached certifi-2021.10.8-py2.py3-none-any.whl (149 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  adding: requests-2.27.1.dist-info/ (stored 0%)
  adding: requests-2.27.1.dist-info/INSTALLER (stored 0%)
  adding: requests-2.27.1.dist-info/top_level.txt (stored 0%)
  adding: requests-2.27.1.dist-info/RECORD (deflated 55%)
  adding: requests-2.27.1.dist-info/REQUESTED (stored 0%)
  adding: requests-2.27.1.dist-info/METADATA (deflated 58%)
  adding: requests-2.27.1.dist-info/WHEEL (deflated 14%)
  adding: requests-2.27.1.dist-info/LICENSE (deflated 65%)
  adding: idna/ (stored 0%)
  adding: idna/core.py (deflated 77%)
  adding: idna/compat.py (deflated 40%)
  adding: idna/uts46data.py (deflated 80%)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.1.1 requires botocore<1.23.25,>=1.23.24, but you have botocore 1.23.4 which is incompatible.


### (2) Zip 파일을 S3에 업로드

In [9]:
import os

prefix = 'Lambda-BizLogic-DynamoDB'
file_name = DDBInsertCodeZip

s3_file_path = 's3://' + os.path.join(bucket, project_prefix, prefix, file_name)
local_file_path = os.path.join(DDBInsertFolder, file_name)

print("s3_file_path: ", s3_file_path)
print("local_file_path: ", local_file_path)




s3_file_path:  s3://ee-assets-prod-us-east-1/modules/c16b965656fb4eba8676d6f0ac759300/v1/Lex-KR_BankingBot/Lambda-BizLogic-DynamoDB/DDBInsert.zip
local_file_path:  Deploy-Package/Lambda-BizLogic-DynamoDB/DDBInsert.zip


In [11]:
%%sh -s "{local_file_path}" "{s3_file_path}"

export AWS_ACCESS_KEY_ID=<dummy> # 키를 입력 하세요
export AWS_SECRET_ACCESS_KEY=<dummy> # Secret Key 입력하세요
export AWS_SESSION_TOKEN=<dummy> # Token 입력하세요 
export AWS_DEFAULT_REGION=<dummy> # region 입력 하세요  (예: us-east-1)

local_file_path=$1
s3_file_path=$2

aws s3 cp $local_file_path $s3_file_path 
aws s3 ls $s3_file_path 

upload: Deploy-Package/Lambda-BizLogic-DynamoDB/DDBInsert.zip to s3://ee-assets-prod-us-east-1/modules/c16b965656fb4eba8676d6f0ac759300/v1/Lex-KR_BankingBot/Lambda-BizLogic-DynamoDB/DDBInsert.zip
2022-02-16 11:22:43     816273 DDBInsert.zip


# 3. 비즈 로직 Banking Service 람다 코드 준비

In [12]:
BizLambdaFolder = "Deploy-Package/Lambda-BizLogic-BankingService"
BizLambdaCodeZip = "BankingServiceFunction.zip"

In [15]:
%%sh -s {BizLambdaFolder} {BizLambdaCodeZip} {Biz_Logic_Code_Folder}
BizLambdaFolder=$1
BizLambdaCodeZip=$2
Biz_Logic_Code_Folder=$3

echo $BizLambdaFolder
echo $BizLambdaCodeZip
echo $Biz_Logic_Code_Folder

# 코드 있는 폴더로 이동
cd $BizLambdaFolder

# 코드를 압축한 zip 파일 있으면 삭제
if test -f $BizLambdaCodeZip; then
    rm $BizLambdaCodeZip
    echo "Existing Code Zip file is deleted"
fi

echo 'Current Directory: '
echo $PWD 

# 코드 파일 및 데이터 파일을 압축 파일에 추가
zip -g -j $BizLambdaCodeZip ../../$Biz_Logic_Code_Folder/*.js




Deploy-Package/Lambda-BizLogic-BankingService
BankingServiceFunction.zip
Artifact/Lambda-BizLogic-BankingService-Code
Existing Code Zip file is deleted
Current Directory: 
/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/aiservices/lex-korean-workshop/Create-CF-Lex-KR-Workshop/Deploy-Package/Lambda-BizLogic-BankingService
	zip warning: BankingServiceFunction.zip not found or empty
  adding: banking_service.js (deflated 66%)
  adding: check_balance.js (deflated 65%)
  adding: fallback.js (deflated 55%)
  adding: index.js (deflated 62%)
  adding: transfer.js (deflated 78%)
  adding: utils.js (deflated 82%)


In [16]:
import os

prefix = 'Lambda-BizLogic-BankingService'
file_name = "BankingServiceFunction.zip"

s3_file_path = 's3://' + os.path.join(bucket, project_prefix, prefix, file_name)
local_file_path = os.path.join(BizLambdaFolder, file_name)

print("s3_file_path: ", s3_file_path)
print("local_file_path: ", local_file_path)




s3_file_path:  s3://ee-assets-prod-us-east-1/modules/c16b965656fb4eba8676d6f0ac759300/v1/Lex-KR_BankingBot/Lambda-BizLogic-BankingService/BankingServiceFunction.zip
local_file_path:  Deploy-Package/Lambda-BizLogic-BankingService/BankingServiceFunction.zip


In [18]:
%%sh -s "{local_file_path}" "{s3_file_path}"


export AWS_ACCESS_KEY_ID=<dummy> # 키를 입력 하세요
export AWS_SECRET_ACCESS_KEY=<dummy> # Secret Key 입력하세요
export AWS_SESSION_TOKEN=<dummy> # Token 입력하세요 
export AWS_DEFAULT_REGION=<dummy> # region 입력 하세요  (예: us-east-1)


local_file_path=$1
s3_file_path=$2

aws s3 cp $local_file_path $s3_file_path 
aws s3 ls $s3_file_path 

upload: Deploy-Package/Lambda-BizLogic-BankingService/BankingServiceFunction.zip to s3://ee-assets-prod-us-east-1/modules/c16b965656fb4eba8676d6f0ac759300/v1/Lex-KR_BankingBot/Lambda-BizLogic-BankingService/BankingServiceFunction.zip
2022-02-16 11:23:53       5535 BankingServiceFunction.zip


# 4. 렉스 봇 아티펙트 설치 람다 코드 준비

폴더 구조
- request 라이브러리
- bot_details.json
    ```
{
  "name": "CardServices",
  "bot_definition": "CardServicesBot-DRAFT-AIGCF4Y0R2-LexJson.zip"
}
    
    ```
- Bot 익스포트 zip 파일 (에: "CardServicesBot-DRAFT-AIGCF4Y0R2-LexJson.zip")
- cfnresponse.py
- lambda_function.py

### (1) Lex Import ZIP 생성

In [19]:
LexImportFolder = "Deploy-Package/Lambda-LexArtifact-Import"
LexImportCodeZip = "LexImport.zip"

In [20]:
%%sh -s {LexImportFolder} {LexImportCodeZip} {Lex_Artifact_Folder}
LexImportFolder=$1
LexImportCodeZip=$2
Lex_Artifact_Folder=$3

echo $LexImportFolder
echo $LexImportCodeZip
echo $Lex_Artifact_Folder


# 코드 있는 폴더로 이동
cd $LexImportFolder

# 코드를 압축한 zip 파일 있으면 삭제
if test -f $LexImportCodeZip; then
    rm $LexImportCodeZip
    echo "Existing Code Zip file is deleted"
fi

# package 폴더 있으면 삭제
if [ -d "package" ]; then
    rm -rf package
    echo "existing package folder is deleted" 
fi    

# requests 패키지 다운로드
pip install --target ./package requests

cd package
# 패키지 폴더 내용 압축
zip -r ../$LexImportCodeZip .
cd ..

echo 'Current Directory: '
echo $PWD 


# 코드 파일 및 데이터 파일을 압축 파일에 추가
zip -g -j $LexImportCodeZip ../../$Lex_Artifact_Folder/*.py  ../../$Lex_Artifact_Folder/*.json ../../$Lex_Artifact_Folder/*.zip




Deploy-Package/Lambda-LexArtifact-Import
LexImport.zip
Artifact/Lambda-LexArtifact-Export-Artifact
Existing Code Zip file is deleted
existing package folder is deleted
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached urllib3-1.26.8-py2.py3-none-any.whl (138 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Using cached certifi-2021.10.8-py2.py3-none-any.whl (149 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  adding: requests-2.27.1.dist-info/ (stored 0%)
  adding: requests-2.27.1.dist-info/INSTALLER (stored 0%)
  adding: requests-2.27.1.dist-info/top_level.txt (stored 0%)
  adding: requests-2.27.1.dist-info/RECORD (deflated 55%)
  adding: requests-2.27.1.dist-info/REQUESTED (stored 0%)
  adding: requests-2.27.1.dist-info/METADATA (deflated 58%)
  adding: requests-2.27.1.dist-info/WHEEL (deflated 14%)
  adding: requests-2.27.1.dist-info/LICENSE (deflated 65%)
  adding: idna/ (stored 0%)
  adding: idna/core.py (deflated 77%)
  addin

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.1.1 requires botocore<1.23.25,>=1.23.24, but you have botocore 1.23.4 which is incompatible.


In [21]:
import os

prefix = 'Lambda-LexArtifact-Import'
file_name = LexImportCodeZip

s3_file_path = 's3://' + os.path.join(bucket, project_prefix, prefix, file_name)
local_file_path = os.path.join(LexImportFolder, file_name)

print("s3_file_path: ", s3_file_path)
print("local_file_path: ", local_file_path)



s3_file_path:  s3://ee-assets-prod-us-east-1/modules/c16b965656fb4eba8676d6f0ac759300/v1/Lex-KR_BankingBot/Lambda-LexArtifact-Import/LexImport.zip
local_file_path:  Deploy-Package/Lambda-LexArtifact-Import/LexImport.zip


In [22]:
%%sh -s "{local_file_path}" "{s3_file_path}"


export AWS_ACCESS_KEY_ID=<dummy> # 키를 입력 하세요
export AWS_SECRET_ACCESS_KEY=<dummy> # Secret Key 입력하세요
export AWS_SESSION_TOKEN=<dummy> # Token 입력하세요 
export AWS_DEFAULT_REGION=<dummy> # region 입력 하세요  (예: us-east-1)



local_file_path=$1
s3_file_path=$2

aws s3 cp $local_file_path $s3_file_path 
aws s3 ls $s3_file_path 

upload: Deploy-Package/Lambda-LexArtifact-Import/LexImport.zip to s3://ee-assets-prod-us-east-1/modules/c16b965656fb4eba8676d6f0ac759300/v1/Lex-KR_BankingBot/Lambda-LexArtifact-Import/LexImport.zip
2022-02-16 11:24:32     825562 LexImport.zip
